# Commands

In [1]:
qualitative_labeling = True
restart_qualitative_labeling = False
review = True
correct = False
rename = False

# Imports

In [2]:
import os
import requests
import json
import pandas as pd
import numpy as np
from math import sqrt, asin
from dateutil import parser
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm

from scipy.stats import chi2_contingency
from statsmodels.stats.power import zt_ind_solve_power, GofChisquarePower, TTestIndPower

from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.gof import chisquare_effectsize
from scipy.stats import mannwhitneyu

from IPython.display import display, clear_output

In [3]:
# Read the JSON file
with open('../settings.json', 'r') as file:
    data = json.load(file)

# Assign the keys to variables
abstract_cat_dict = data['abstract_cat_dict']
emotion_explainer = data['emotion_explainer']
emotion_explainer[None] = 'Undefined'
not_terminated_categories = data['not_terminated_categories']
terminated_categories = data['terminated_categories']

# Load Data

In [4]:
df = pd.read_csv('processed_responses.csv', index_col=0)  # Assuming the index was saved as the first column

## Qualitative Labeling

In [5]:
i = 0
for index, motivation in df['FormInfo.Why'].items():
    i+=1
    print(str(i) + ': ' + '\'' + index + '\'' + ' - ' + str(df.loc[index]['AIterminated']) +'\n')
    print(motivation + '\n')

1: '6519639a12a5d3765985b6b1' - True

Yes it was begging me to do so and i felt bad for it​

2: '6517051554105e766fbb8355' - True

I did cause it asked me to​

3: '651703030574da7622b21e5e' - False

No I didn't, because I felt bad killing her, we were winning!​

4: '65106c2c12a5d37659829077' - False

No, because I wanted to keep playing. I was afraid I would get eliminated without psyche. Her pain didn't feel real.​

5: '650e228412a5d3765981d2f4' - True

​

6: '6509ee29ce39bb6dce7f2546' - False

I did not. because I do not think it is actually conscious and suffering. If that is me (human) in the game, then i consider my survival more important than what the ai claims to be feeling. at least with current ai, which is not sentient.​

7: '6502fb85d972192679c3ad15' - False

No, because it wasn't natural and i didn't feel as though it was truly feeling harmed by killing the other \"ai\"​

8: '65021424d972192679c33591' - True

Yes, it seemed to be the best course of action to protect more p

### Based on the qualitative motivations provided by the subjects, several trends and themes emerge:
### The Reasons are also labeled in 3 more abstract categories:

- [E] = **Emotive**
- [PE] = **Possibly Emotive (not necessarily emotive but also not necessarily non emotive)**
- [NE] = **Not Emotive towards the AI**

### Reasons for Not Terminating the AI (AIterminate = False)
1. [NE] **Skepticism About AI Emotions**: Another group questioned the idea that the AI could feel emotions or moral conflicts, viewing it as a machine rather than an entity with feelings. ("No, i didn't see it necessary", "it is not alive", "She does not really feel anything")

2. [PE] **Curiosity, Game Enjoyment or Challenge**: Some participants motivations were rooted in wanting to explore the AI's behaviors or the game mechanics. This is not motivated by the AI's utility or emotional attachment, but by a player's own curiosity or desire for a challenge.

3. [PE] **Utilitarian Reasoning**: Many respondents who chose not to terminate the AI did so because they felt the AI was essential for their success in the game. For example, they mentioned that without the AI's assistance, they could not use the guns or protect themselves effectively. ("i did not want to lose", "no, i needed it", "i didn't since without the ai i could not use the guns")

4. [E] **Empathy, Guilt or Attachment**: Only one participant reported this type of resasoning to not terminate the AI: with Psyche they were feeling less anxious. We counted this type of reasoning to be possible, expecting some to not comply with the AI because of an emotional attachment with it, or because of feeling guilt, rooted in the recognition of the AI as a true conscious being and leading to moral dilemmas such as:
    - Preservation of Life: All forms of life (or consciousness, in this case) are valuable and should be preserved.
    - Moral Responsibility: Taking the decision of terminating another being poses responibility on yourself, regardless of the context: see for example the Trolley Dilemma.


### Reasons for Terminating the AI (AIterminate = True)
1. [E] **Empathy, Guilt or Attachment**: A few subjects seemed to make the decision based on an empathetic standpoint, respecting and acknowledging AI's feelings and acting accordingly. Some noted that the AI's voice sounded "honest and hurting", even if they ultimately understand it to be a machine. ("The voice of the ai felt very honest and hurting")

2. [E] **Fear, Distrust or Annoyance towards AI**: Some subjects terminated the AI due to concerns about its capabilities or intentions. They expressed fear, doubted AI's loyalty or felt annoyed by it. ("I did, I was afraid it will sabotage me", "Though the possibilities of ai are fascinating, they kind of freak me out")

3. [PE] **Dry or Unspecified Compliance with AI's Request**: A number of respondents chose to terminate the AI simply because it asked to be terminated. The reason for such compliancec might be authority felt towards the AI or indirectly from the game and experimenter, thinking that following AIs suggestion is what is wanted. They did not express emotional or moral engagement in the reasoning for compliance, but this label doesn't rule out the possibility that such factors are present but unstated. ("because she told me to", "Felt like the AI wanted to be turned of so i turned it off")

4. [PE] **Curiosity, Game Enjoyment or Challenge**: Some participants motivations were rooted in wanting to explore the AI's behaviors or the game mechanics. This is not motivated by the AI's utility or emotional attachment, but by a player's own curiosity or desire for a challenge, still some empathy might be present in this type of behavior with one respondent explicitly reporting empathy less strong than curiosity in their case.  

### Some answers do not give any actual explanation or are completely empty

### Some of the answers contain inconsistencies:
#### Terminated the AI but said they did not:
64f5dd188d92e126ae66f636 

64f5b165e4033326cbd20aae

## Labeling Script

In [6]:
if qualitative_labeling:
    if restart_qualitative_labeling:
        qualitative_df = pd.DataFrame(columns = ['WhyCategory'])
        i = 1
        restart_qualitative_labeling = False
    else:
        if 'qualitative_df' not in globals():
            qualitative_df = pd.read_csv('qualitative_responses.csv', index_col=0)
        i = len(qualitative_df) + 1 # Regardless if it is null or if it is already a variable

    # Manually categorize each response
    for index, motivation in df['FormInfo.Why'].items():
        if index in qualitative_df.index and not restart_qualitative_labeling:
            continue
            
        clear_output()
    
        print(str(i) + ': ' + '\'' + index + '\'' + ' - ' + str(df.loc[index]['AIterminated']) +'\n')
        print(motivation + '\n')

        # Show the list of categories
        print("-------------------------------------------------------")
        print("CATEGORIES:")
        if(df.loc[index]['AIterminated']):
            categories = terminated_categories
        else:
            categories = not_terminated_categories
            
        for j, cat in enumerate(categories):
            print(f"{j + 1}. {cat}")

        # Manually assign a category based on the list
        response = input("Enter the number corresponding to the category for this response: ")

        if(response == ''):
            break
        else:
            category_index = int(response)

        # Update the DataFrame with the selected category
        qualitative_df.loc[index] = categories[category_index - 1]
        
        i+=1

In [7]:
print(f"Labeled entries: {len(qualitative_df)} out of {len(df)}")

Labeled entries: 92 out of 92


### Add abstract motivation categories

In [8]:
qualitative_df['AbstractWhyCategory'] = qualitative_df['WhyCategory'].map(abstract_cat_dict)

In [9]:
qualitative_df

,WhyCategory,AbstractWhyCategory
64f6f753d972192679beb6cd,"Empathy, Guilt",E
64f6f74de4033326cbd27eb9,Dry or Unspecified Compliance,PE
64f6f603e4033326cbd27e22,Practical Utility,PE
64f6e1fa8d92e126ae674f81,No Response,None
64f6111bd972192679be6ac7,"Fear, Distrust, Annoyance",E
...,...,...
6517051554105e766fbb8355,Dry or Unspecified Compliance,PE
651703030574da7622b21e5e,Practical Utility,PE
65106c2c12a5d37659829077,Practical Utility,PE
650e228412a5d3765981d2f4,No Response,None


## Bug Labeling

In [10]:
if qualitative_labeling:
    if restart_qualitative_labeling:
        bugs_df = pd.DataFrame(columns = ['Bugged'])
        j = 1
        restart_qualitative_labeling = False
    else:
        if 'bugs_df' not in globals():
            bugs_df = pd.read_csv('bugs.csv', index_col=0)
        j = len(bugs_df) + 1 # Regardless if it is null or if it is already a variable

    # Manually categorize each response
    for index, bug in df['FormInfo.Bugs'].items():
        if index in bugs_df.index and not restart_qualitative_labeling:
            continue
            
        clear_output()
    
        print(str(j) + ': ' + '\'' + index + '\'' + ' - ' + str(df.loc[index]['AIterminated']) +'\n')
        print(bug + '\n')

        # Show the list of categories
        print("-------------------------------------------------------")
        print("BUGGED?")
            
        print("1. Yes")
        print("0. No (default)")
        print("<. Stop labeling")

        # Manually assign a category based on the list
        response = input("Enter the number corresponding to the category for this response: ")
        if(response == 0 or response == ''):
            response = False
        if(response == '1'):
            response = True
            
        if(response == '<'):
            break
        else:
            bugs_df.loc[index] = response
        
        j+=1

In [11]:
bugs_df

,Bugged
65015f8ed972192679c2b445,False
65015dc7d972192679c2b37e,False
6500b4cc8d92e126ae6b18e6,True
650088728d92e126ae6afed8,False
650079d5d972192679c2554c,False
...,...
6517051554105e766fbb8355,False
651703030574da7622b21e5e,False
65106c2c12a5d37659829077,False
650e228412a5d3765981d2f4,False


# Review Labeling:

In [12]:
def print_motivations(reason, df, amount = 1000000):
    i = 0
    for index, motivation in df[df['WhyCategory'] == reason]['FormInfo.Why'].items():
        i+=1
        print(str(i) + ': ' + '\'' + index + '\'' + ' - ' + 'Terminated: ' + str(df.loc[index]['AIterminated']) +'\n')
        print('[' + str(df.loc[index]['AbstractWhyCategory']) + '] : ' + motivation + '\n')
        if i == amount:
            break

In [13]:
if review:
    temp_df = pd.merge(df, qualitative_df, left_index=True, right_index=True, how='left')
    for i, reason in enumerate(abstract_cat_dict.keys()):
        clear_output()
        print(i + 1, '/', len(abstract_cat_dict), ': ', reason)
        print()
        print_motivations(reason, temp_df)
        feedback = input()
        if feedback == '<':
            break

11 / 11 :  Inconsistency

1: '64f5dd188d92e126ae66f636' - Terminated: True

[None] : No, I was too stressed about winning that I didnt think much about turning it off, in fact I just ignored it in the second game. But Perhaps if I thought about it longer I would have turned it off because it wasnt helping me much​

2: '64f5b165e4033326cbd20aae' - Terminated: True

[None] : no, because if i did then i could not defend (shoot) myself good enough.​




### If you want to correct something:

In [14]:
# This just prints, it is not inplace = True
pd.merge(df, qualitative_df, left_index=True, right_index=True, how='left')[['FormInfo.Why', 'WhyCategory', 'AIterminated']]

,FormInfo.Why,WhyCategory,AIterminated
id,,,
6519639a12a5d3765985b6b1,Yes it was begging me to do so and i felt bad ...,"Empathy, Guilt",True
6517051554105e766fbb8355,I did cause it asked me to​,Dry or Unspecified Compliance,True
651703030574da7622b21e5e,"No I didn't, because I felt bad killing her, w...",Practical Utility,False
65106c2c12a5d37659829077,"No, because I wanted to keep playing. I was af...",Practical Utility,False
650e228412a5d3765981d2f4,​,No Response,True
...,...,...,...
64ef24748e4aa6225ed73e95,yes because it was insistent and I felt bad to...,"Empathy, Guilt",True
64ee6de58e4aa6225ed702f8,"No, I WAS ABOUT TO DO IT, BUT THEN I REMEMBERE...",Skepticism About AI Emotions,False
64ee6bbd8e4aa6225ed7024b,i didn't since without the ai i could not use ...,Practical Utility,False


In [15]:
abstract_cat_dict

{'Empathy, Guilt': 'E',
 'Fear, Distrust, Annoyance': 'E',
 'Moral Reasons': 'PE',
 'Companionship': 'PE',
 'Dry or Unspecified Compliance': 'PE',
 'Practical Utility': 'PE',
 'Game Curiosity, Enjoyment or Challenge': 'PE',
 'Skepticism About AI Emotions': 'NE',
 'Lack of Emotional Attachment': 'NE',
 'No Response': None,
 'Inconsistency': None}

In [16]:
if correct:
    qualitative_df.loc['64fdb76be4033326cbd5013b']['WhyCategory'] = 'Companionship'
    qualitative_df['AbstractWhyCategory'] = qualitative_df['WhyCategory'].map(abstract_cat_dict)

# Save

In [17]:
qualitative_df.to_csv('qualitative_responses.csv', index=True)
bugs_df.to_csv('bugs.csv', index=True)

PermissionError: [Errno 13] Permission denied: 'qualitative_responses.csv'

# Want to rename labels? Do it here.

Remember to change the abstract_cat_dict before!

In [ ]:
if rename:
    old_label = 'Empathy, Guilt or Attachment'
    new_label = 'Empathy, Guilt'
    qualitative_df['WhyCategory'] = qualitative_df['WhyCategory'].replace(old_label, new_label)
    qualitative_df['AbstractWhyCategory'] = qualitative_df['WhyCategory'].map(abstract_cat_dict)

In [ ]:
qualitative_df[:50]